In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image
import numpy as np
import json
from skimage.draw import line
from geographiclib.geodesic import Geodesic
import math

In [37]:
class Weather():
    def __init__(self, weather_img = "weather.png" , weather_json = "weather.json"):
        
        self.weather_matrix = self.to_matrix(weather_img).astype(np.int8)
        self.bottom_right, self.top_left = self.coordinates(weather_json)
        
    def coordinates(self, weather_json):
        weather_coordinates = json.load(open('weather.json'))
        
        return [weather_coordinates["bottom_right"]["latitude"], weather_coordinates["bottom_right"]["longitude"]] \
                 , [ weather_coordinates["top_left"]["latitude"], weather_coordinates["top_left"]["longitude"]]
    def br(self):
        return self.bottom_right
    def tl(self):
        return self.top_left
    
    def to_matrix(self, weather_img):
        
        weather_matrix = np.array(matplotlib.image.imread(weather_img)[: ,: , 1])
        un = np.unique(weather_matrix[0], axis = 0)
        
        def map_green(x):
            return (np.isclose(x , un[1]))
        def map_yellow(x):
            return (np.isclose(x , un[2]))
        
        return weather_matrix*0 + map_green(weather_matrix)*1 + map_yellow(weather_matrix)*2
    
    def wmatrix(self):
        
        return self.weather_matrix
    
    def weight_between_points(self, p1, p2):
        
        assert (abs(self.bottom_right[0]) < abs(p1[0]) and abs(p1[0]) < abs(self.top_left[0])) and (abs(self.bottom_right[1]) < abs(p1[1]) and abs(p1[1]) < abs(self.top_left[1]))
        assert (abs(self.bottom_right[0]) < abs(p2[0]) and abs(p2[0]) < abs(self.top_left[0])) and (abs(self.bottom_right[1]) < abs(p2[1]) and abs(p2[1]) < abs(self.top_left[1]))
        
        p1_idx_x, p1_idx_y = self.coordinates_to_pixel_idx(p1)
        p2_idx_x, p2_idx_y = self.coordinates_to_pixel_idx(p2)

        weight_of_pixels_in_line = self.weather_matrix[line(p1_idx_x, p1_idx_y, p2_idx_x, p2_idx_y)]
                
        val_counts =  np.array(np.unique(weight_of_pixels_in_line, return_counts = True))
        
        weather = [0, 0, 0]
        for i in range(0, len(val_counts[0])):
            weather[val_counts[0][i]] = val_counts[1][i] 
        
        return np.array(weather)
        
    def coordinates_to_pixel_idx(self, point):
        
        x_array = np.arange(self.top_left[0], self.bottom_right[0], (self.bottom_right[0]-self.top_left[0])/self.weather_matrix.shape[0])
        y_array = np.arange(self.top_left[1], self.bottom_right[1], (self.bottom_right[1]-self.top_left[1])/self.weather_matrix.shape[1])

        return  np.abs(x_array - point[0]).argmin(), np.abs(y_array - point[1]).argmin()

In [231]:
class Projection:
    def __init__(self, weather_json_coordinates = "weather.json", weather_img = "weather.png"):
        self.geod = Geodesic.WGS84
        self.bottom_right, self.top_left = self.coordinates(weather_json_coordinates)
        self.weather_matrix_shape = np.array(matplotlib.image.imread(weather_img)).shape
        
        self.x_grid = np.arange(self.top_left[0], self.bottom_right[0], (self.bottom_right[0]-self.top_left[0])/self.weather_matrix_shape[0])
        self.y_grid = np.arange(self.top_left[1], self.bottom_right[1], (self.bottom_right[1]-self.top_left[1])/self.weather_matrix_shape[1])
    
    
    
    def coordinates(self, weather_json):
        weather_coordinates = json.load(open('weather.json'))
        
        return [weather_coordinates["bottom_right"]["latitude"], weather_coordinates["bottom_right"]["longitude"]] \
                 , [ weather_coordinates["top_left"]["latitude"], weather_coordinates["top_left"]["longitude"]]
    
    def distance(self, lat1, lon1, lat2, lon2):
        return self.geod.Inverse(lat1, lon1, lat2, lon2, Geodesic.DISTANCE)['s12']/1000
    
    def waypoints(self, lat1, lon1, lat2, lon2, ds = 60e3):
        line = self.geod.InverseLine(lat1, lon1, lat2, lon2)
        n_steps = int(math.ceil(line.s13 / ds))
                      
        coordinates_list_lat = []
        coordinates_list_lon = []

        for i in range(n_steps + 1):
            s = min(ds * i, line.s13)
            g = line.Position(s, Geodesic.LATITUDE | Geodesic.LONGITUDE | Geodesic.LONG_UNROLL)
            coordinates_list_lat.append(g['lat2'])
            coordinates_list_lon.append(g['lon2'])
            
        return np.array(coordinates_list_lat) , np.array(coordinates_list_lon)
    
    def waypoints_as_idx(self, lat1, lon1, lat2, lon2, ds = 60e3):

        waypoints_x_idx ,waypoints_y_idx = self.waypoints( lat1, lon1, lat2, lon2, ds)
        
        for i in range(0, len(waypoints_x_idx)):
            waypoints_x_idx[i] = np.abs(self.x_grid - waypoints_x_idx[i]).argmin()
            waypoints_y_idx[i] = np.abs(self.y_grid - waypoints_y_idx[i]).argmin()
        
        return waypoints_x_idx, waypoints_y_idx
    
    def shortest_path_as_idx(self, lat1, lon1, lat2, lon2, ds = 60e3):
        

    def coordinates_to_pixel_idx(self, point):
        
        x_array = np.arange(self.top_left[0], self.bottom_right[0], (self.bottom_right[0]-self.top_left[0])/self.weather_matrix_shape[0])
        y_array = np.arange(self.top_left[1], self.bottom_right[1], (self.bottom_right[1]-self.top_left[1])/self.weather_matrix_shape[1])

        return  np.abs(x_array - point[0]).argmin(), np.abs(y_array - point[1]).argmin()

In [232]:
cord = Projection()

In [233]:
cord.distance(35.043794, -106.816311, 34.702697, -95.864267)

1001.5433136544921

In [234]:
cord.waypoints_as_idx(35.043794, -106.816311, 34.702697, -95.864267)

(array([784., 784., 784., 784., 784., 784., 785., 785., 786., 787., 788.,
        789., 790., 791., 793., 794., 796., 797.]),
 array([ 748.,  766.,  783.,  801.,  818.,  836.,  853.,  870.,  888.,
         905.,  923.,  940.,  958.,  975.,  992., 1010., 1027., 1039.]))

In [235]:
cord.waypoints(35.043794, -106.816311, 34.702697, -95.864267)

(array([35.043794  , 35.0512608 , 35.05516282, 35.05549922, 35.05226991,
        35.0454756 , 35.03511776, 35.02119865, 35.00372129, 34.98268945,
        34.9581077 , 34.92998134, 34.89831643, 34.86311981, 34.82439902,
        34.78216235, 34.73641884, 34.702697  ]),
 array([-106.816311  , -106.15873269, -105.50106325, -104.84335983,
        -104.1856796 , -103.52807973, -102.87061729, -102.21334925,
        -101.55633242, -100.89962336, -100.24327838,  -99.58735348,
         -98.93190428,  -98.27698601,  -97.62265341,  -96.96896074,
         -96.3159617 ,  -95.864267  ]))

In [ ]:
l = geod.InverseLine(35.043794, -106.816311, 33.371758, -96.361231)
ds = 60e3; n = int(math.ceil(l.s13 / ds))
for i in range(n + 1):
    if i == 0:
        print("distance latitude longitude azimuth")
    s = min(ds * i, l.s13)
    g = l.Position(s, Geodesic.LATITUDE | Geodesic.LONGITUDE )
    print("{:.0f} {:.5f} {:.5f}".format(
    g['s12'], g['lat2'], g['lon2']))

In [44]:
w = Weather()

In [6]:
#plt.figure(figsize = (20, 20))
#plt.imshow(w.wmatrix())

In [7]:
br = w.br()

In [8]:
tl = w.tl()

In [9]:
br

[21.94, -67.5]

In [10]:
tl

[55.78, -135.0]

In [11]:
w.weight_between_points([35.043794, -106.816311], [33.371758, -96.361231])

array([140,  90,  49])

In [22]:
weather_matrix = w.wmatrix()

In [28]:
x_array = np.arange(tl[0], br[0], (br[0]-tl[0])/weather_matrix.shape[0])
y_array = np.arange(tl[1], br[1], (br[1]-tl[1])/weather_matrix.shape[0])

In [29]:
x_array

array([55.78     , 55.7535625, 55.727125 , ..., 22.0193125, 21.992875 ,
       21.9664375])

In [30]:
(br[0]-tl[0])/weather_matrix.shape[0]

-0.026437500000000003

In [31]:
(br[1]-tl[1])/weather_matrix.shape[0]

0.052734375

In [32]:
y_array

array([-135.        , -134.94726562, -134.89453125, ...,  -67.65820312,
        -67.60546875,  -67.55273438])